Load packages

In [1]:
import sys
import time
from datetime import datetime, timedelta

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math as mt
from math import sin, cos, pi  # rotating regions
from math import floor  # truncating naics codes
import numba  # speed up data transform with JIT compilation

First mount google drive

In [2]:
# Root directory for dataset

dataroot = # where you save and load data


Define Constants

In [11]:
# "grid coordinates" created in R are in meters
cell_width = 1 * 0.025 * 1.609344 * 1000  # cell width in meters (convert from miles)
size_potential = 10  # potential locations: num_width_potential x num_width_potential
size_padding = 20  # number of padding cells on each side of potential grid
nc = 25 #3  # number of channels

BATCH_SIZE_real = 96  # regions with missing grocery store per batch #96
BATCH_SIZE_fill = 16 #16  # regions with real location filled in (-> no missing) per batch
BATCH_SIZE_random = 8  # random regions (-> no missing) per batch
BATCH_SIZE = BATCH_SIZE_real + BATCH_SIZE_fill + BATCH_SIZE_random
# probability that a training region has no unrealized location
# prob_mult = 80  # how many times more likely is no unrealized location
# prob_none = prob_mult / (prob_mult + pow(size_potential,2))

frac_train_real = 1.0  # fraction of real regions to use for training
frac_train_random = 1.0  # fraction of random (unrealized) regions to use for training
# remaining regions used for out-of-sample evaluation
# if frac_... = 1, the full sample is used for training AND evaluation
# TODO: implement frac_... such that eval regions are non-overlapping with train regions

use_saved_model = False #True
saved_model_filename = 'classifier-checkpoint-2022-08-27--20-06.tar'

use_cuda = True

EPOCHS = 10
sampleTotNum = 418
ITERS = 100 #20*mt.ceil(sampleTotNum/BATCH_SIZE_real) #30*mt.ceil(sampleTotNum/BATCH_SIZE) #100

labelThresh = 1000 # start from where there are fake openings

weight_fine = 1
weight_plain = 1

eval_weight_fine = 1
eval_weight_plain = 1

fineSampleNumber = 418

In [12]:
print('BATCH_SIZE: ' + str(BATCH_SIZE))
print('cell_width: ' + str(round(cell_width)) + 'm')
# print('prob_none: ' + str(round(100 * prob_none)) + '%')

BATCH_SIZE: 120
cell_width: 40m


Read in data from google drive

In [13]:
dict_S_I = dict()
dict_S_I_restaurant = dict()

# read in data of businesses near each grocery store
with open(dataroot+'grid_S_I.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        i_id = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])

        if slist[4] == 'NA':
            popu = -1
        else:
            popu = float(slist[4])

        if slist[5] == 'NA':
            sau16 = -1
        else:
            sau16 = float(slist[5])

        if slist[6] == 'NA':
            sa1664 = -1
        else:
            sa1664 = float(slist[6])
                    
        if slist[7] == 'NA':
            sw = -1
        else:
            sw = float(slist[7])
                    
        if slist[8] == 'NA':
            sc = -1
        else:
            sc = float(slist[8])
                    
        if slist[9] == 'NA':
            scp = -1
        else:
            scp = float(slist[9])
                    
        if slist[10] == 'NA':
            mhh = -1
        else:
            mhh = float(slist[10])
                    
        if slist[11] == 'NA':
            suem = -1
        else:
            suem = float(slist[11])
                    
        if slist[12] == 'NA':
            mhhv = -1
        else:
            mhhv = float(slist[12])
                    
        if slist[13] == 'NA':
            stc = -1
        else:
            stc = float(slist[13])
                    
        if slist[14] == 'NA':
            stp = -1
        else:
            stp = float(slist[14])
                    
        if slist[15] == 'NA':
            sth = -1
        else:
            sth = float(slist[15])
                    
        if slist[16] == 'NA':
            st15 = -1
        else:
            st15 = float(slist[16])
                    
        if slist[17] == 'NA':
            st30 = -1
        else:
            st30 = float(slist[17])
                    
        if slist[18] == 'NA':
            st45 = -1
        else:
            st45 = float(slist[18])
                    
        if slist[19] == 'NA':
            st60 = -1
        else:
            st60 = float(slist[19])
                    
        if slist[20] == 'NA':
            st90 = -1
        else:
            st90 = float(slist[20])
                    
        #if slist[21] == 'NA':
        #    naics = -1
        #else:
        #    naics = int(float(slist[21]))
                    
        if slist[22] == 'NA':
            minr = -1
        else:
            minr = float(slist[22])
        
        tup = (x,y,popu, sau16, sa1664, sw, sc, scp, mhh, suem, 
               mhhv, stc, stp, sth, st15, st30, st45, st60, st90, minr)
        #print(tup)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I.keys():
            dict_S_I[s_id] = list()
        # add data to this s_id
        dict_S_I[s_id].append(tup)

        '''
        if floor(naics / 100) == 7225:
            # create entry if first time we encounter s_id
            if not s_id in dict_S_I_restaurant.keys():
                dict_S_I_restaurant[s_id] = list()
            # add data to this s_id
            dict_S_I_restaurant[s_id].append(tup)
        '''

# read in data of businesses near each grocery store
with open(dataroot+'grid_S_random_I.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        i_id = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])


        if slist[4] == 'NA':
            popu = -1
        else:
            popu = float(slist[4])

        if slist[5] == 'NA':
            sau16 = -1
        else:
            sau16 = float(slist[5])

        if slist[6] == 'NA':
            sa1664 = -1
        else:
            sa1664 = float(slist[6])
                    
        if slist[7] == 'NA':
            sw = -1
        else:
            sw = float(slist[7])
                    
        if slist[8] == 'NA':
            sc = -1
        else:
            sc = float(slist[8])
                    
        if slist[9] == 'NA':
            scp = -1
        else:
            scp = float(slist[9])
                    
        if slist[10] == 'NA':
            mhh = -1
        else:
            mhh = float(slist[10])
                    
        if slist[11] == 'NA':
            suem = -1
        else:
            suem = float(slist[11])
                    
        if slist[12] == 'NA':
            mhhv = -1
        else:
            mhhv = float(slist[12])
                    
        if slist[13] == 'NA':
            stc = -1
        else:
            stc = float(slist[13])
                    
        if slist[14] == 'NA':
            stp = -1
        else:
            stp = float(slist[14])
                    
        if slist[15] == 'NA':
            sth = -1
        else:
            sth = float(slist[15])
                    
        if slist[16] == 'NA':
            st15 = -1
        else:
            st15 = float(slist[16])
                    
        if slist[17] == 'NA':
            st30 = -1
        else:
            st30 = float(slist[17])
                    
        if slist[18] == 'NA':
            st45 = -1
        else:
            st45 = float(slist[18])
                    
        if slist[19] == 'NA':
            st60 = -1
        else:
            st60 = float(slist[19])
                    
        if slist[20] == 'NA':
            st90 = -1
        else:
            st90 = float(slist[20])
                    
        #if slist[21] == 'NA':
        #    naics = -1
        #else:
        #    naics = int(float(slist[21]))
                    
        if slist[22] == 'NA':
            minr = -1
        else:
            minr = float(slist[22])
        
        tup = (x,y,popu, sau16, sa1664, sw, sc, scp, mhh, suem, 
               mhhv, stc, stp, sth, st15, st30, st45, st60, st90, minr)
        #print(tup)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I.keys():
            dict_S_I[s_id] = list()
        # add data to this s_id
        dict_S_I[s_id].append(tup)
        '''
        if floor(naics / 100) == 7225:
            # create entry if first time we encounter s_id
            if not s_id in dict_S_I_restaurant.keys():
                dict_S_I_restaurant[s_id] = list()
            # add data to this s_id
            dict_S_I_restaurant[s_id].append(tup)
        '''
#dict_S_S = dict()
dict_S_I_cate1 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate1.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate1.keys():
            dict_S_I_cate1[s_id] = list()
        # add data to this s_id
        dict_S_I_cate1[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate1.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate1.keys():
            dict_S_I_cate1[s_id] = list()
        # add data to this s_id
        dict_S_I_cate1[s_id].append(tup)

#dict_S_S = dict()
dict_S_I_cate2 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate2.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate2.keys():
            dict_S_I_cate2[s_id] = list()
        # add data to this s_id
        dict_S_I_cate2[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate2.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate2.keys():
            dict_S_I_cate2[s_id] = list()
        # add data to this s_id
        dict_S_I_cate2[s_id].append(tup)

#dict_S_S = dict()
dict_S_I_cate3 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate3.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate3.keys():
            dict_S_I_cate3[s_id] = list()
        # add data to this s_id
        dict_S_I_cate3[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate3.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate3.keys():
            dict_S_I_cate3[s_id] = list()
        # add data to this s_id
        dict_S_I_cate3[s_id].append(tup)

#dict_S_S = dict()
dict_S_I_cate4 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate4.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate4.keys():
            dict_S_I_cate4[s_id] = list()
        # add data to this s_id
        dict_S_I_cate4[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate4.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate4.keys():
            dict_S_I_cate4[s_id] = list()
        # add data to this s_id
        dict_S_I_cate4[s_id].append(tup)
#dict_S_S = dict()
dict_S_I_cate5 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate5.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate5.keys():
            dict_S_I_cate5[s_id] = list()
        # add data to this s_id
        dict_S_I_cate5[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate5.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate5.keys():
            dict_S_I_cate5[s_id] = list()
        # add data to this s_id
        dict_S_I_cate5[s_id].append(tup)

#dict_S_S = dict()
dict_S_I_comp = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_comp.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_comp.keys():
            dict_S_I_comp[s_id] = list()
        # add data to this s_id
        dict_S_I_comp[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_comp.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_comp.keys():
            dict_S_I_comp[s_id] = list()
        # add data to this s_id
        dict_S_I_comp[s_id].append(tup)




dict_S_I_mat = dict()
for key, value in dict_S_I.items():
   
    # create a numpy array
    mat = np.empty((len(value), 20)) # 20 = 2 + 18
    for i in range(len(value)):
        #print(i)
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
        mat[i,2] = value[i][2]
        mat[i,3] = value[i][3]
        mat[i,4] = value[i][4]
        #print(value[i][4])
        mat[i,5] = value[i][5]
        mat[i,6] = value[i][6]
        mat[i,7] = value[i][7]
        mat[i,8] = value[i][8]
        mat[i,9] = value[i][9]
        mat[i,10] = value[i][10]
        mat[i,11] = value[i][11]
        mat[i,12] = value[i][12]
        mat[i,13] = value[i][13]
        mat[i,14] = value[i][14]
        mat[i,15] = value[i][15]
        mat[i,16] = value[i][16]
        mat[i,17] = value[i][17]
        mat[i,18] = value[i][18]
        mat[i,19] = value[i][19]
        
    dict_S_I_mat[key] = mat
    

dict_S_S = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_S.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_oth = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_S.keys():
            dict_S_S[s_id] = list()
        # add data to this s_id
        dict_S_S[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_S.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_oth = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_S.keys():
            dict_S_S[s_id] = list()
        # add data to this s_id
        dict_S_S[s_id].append(tup)

'''
dict_S_I_restaurant_mat = dict()
for key, value in dict_S_I_restaurant.items():
    # create a numpy array
    #print(value[1],len(value))
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_restaurant_mat[key] = mat
# regions without restaurants:
for key in set(dict_S_I.keys()) - set(dict_S_I_restaurant.keys()):
    mat = np.empty((0,2))
    dict_S_I_restaurant_mat[key] = mat
'''

#dict_S_S_mat = dict()
dict_S_S_mat = dict()
for key, value in dict_S_S.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_S_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_S.keys()):
    mat = np.empty((0,2))
    dict_S_S_mat[key] = mat
    

dict_S_I_cate1_mat = dict()
for key, value in dict_S_I_cate1.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate1_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate1.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate1_mat[key] = mat

dict_S_I_cate2_mat = dict()
for key, value in dict_S_I_cate2.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate2_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate2.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate2_mat[key] = mat

dict_S_I_cate3_mat = dict()
for key, value in dict_S_I_cate3.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate3_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate3.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate3_mat[key] = mat

dict_S_I_cate4_mat = dict()
for key, value in dict_S_I_cate4.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate4_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate4.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate4_mat[key] = mat

dict_S_I_cate5_mat = dict()
for key, value in dict_S_I_cate5.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate5_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate5.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate5_mat[key] = mat

dict_S_I_comp_mat = dict()
for key, value in dict_S_I_comp.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_comp_mat[key] = mat
    
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_comp.keys()):
    mat = np.empty((0,2))
    dict_S_I_comp_mat[key] = mat
  

S_id_all = list(dict_S_I.keys())
# print(S_id_all)
S_id_real = [el for el in S_id_all if el<labelThresh]
# print(S_id_real)
S_id_random = [el for el in S_id_all if el>=labelThresh]
# print(S_id_random)

Transform Data to grid (functions)

In [15]:
@numba.jit(nopython=True)
def cnt_in_cell_mat(x,y):
    out = np.zeros((2*size_padding+size_potential,2*size_padding+size_potential))
    for i in range(len(x)):
        if min(x[i],y[i])>= 0 and max(x[i],y[i])<2*size_padding+size_potential:
            out[y[i],x[i]] += 1
    return out

def cnt_in_cell_mat_value(x,y,value):
    out = np.zeros((2*size_padding+size_potential,2*size_padding+size_potential))
    for i in range(len(x)):
        #print('cnt cell',x[i],y[i],value[i],0,2*size_padding+size_potential)
        if min(x[i],y[i])>= 0 and max(x[i],y[i])<2*size_padding+size_potential:
            out[y[i],x[i]] += value[i]
    return out


@numba.jit(nopython=True)
def data_shift_rotate(mat,shift_x=0,shift_y=0,theta=0,mirror_var=1):
    # rotate by theta
    theta = theta * pi / 180
    if not theta == 0:
        x = cos(theta) * mat[:,0] - sin(theta) * mat[:,1]
        y = sin(theta) * mat[:,0] + cos(theta) * mat[:,1]
        # rot = np.array([[cos(theta),sin(theta)],[-sin(theta),cos(theta)]])
        # xy = mat@rot
        # x = xy[:,0]
        # y = xy[:,1]
    else:
        x = mat[:,0]
        y = mat[:,1]
    # mirror the region
    if mirror_var == 1 or mirror_var == -1:
        x = mirror_var * x
    # shift by shift_x, shift_y
    x = x+shift_x
    y = y+shift_y

    return x, y  


def data_to_grid(mat,shift_x=0,shift_y=0,theta=0,mirror_var=1):

    # out = np.zeros((2*size_padding+size_potential,2*size_padding+size_potential))
    x, y = data_shift_rotate(mat,shift_x,shift_y,theta,mirror_var)

    # fit into cells
    x = np.around(x/cell_width + size_padding).astype(int)
    y = np.around(y/cell_width + size_padding).astype(int)

    # for i in range(len(x)):
    #     # count if point is still within the grid
    #     if min(x[i],y[i])>= 0 and max(x[i],y[i])<2*size_padding+size_potential:
    #         out[y[i],x[i]] += 1
    # print(out)
    return cnt_in_cell_mat(x,y)

def data_to_grid_value(mat,shift_x=0,shift_y=0,theta=0,mirror_var=1,valueNo=2):

    # out = np.zeros((2*size_padding+size_potential,2*size_padding+size_potential))
    x, y = data_shift_rotate(mat,shift_x,shift_y,theta,mirror_var)
    #print(x,y)
    value = mat[:,valueNo]
    #print(value)

    # fit into cells
    x = np.around(x/cell_width + size_padding).astype(int)
    y = np.around(y/cell_width + size_padding).astype(int)
    #print(x,y)

    # for i in range(len(x)):
    #     # count if point is still within the grid
    #     if min(x[i],y[i])>= 0 and max(x[i],y[i])<2*size_padding+size_potential:
    #         out[y[i],x[i]] += 1
    # print(out)
    return cnt_in_cell_mat_value(x,y,value)

def create_matLabel(y,x, gratitude = 0.15):

    if gratitude==0:
        out = np.zeros([size_potential,size_potential])
        out[y,x] = 1

        for yi in range(size_potential):
            for xi in range(size_potential):
                out[yi,xi] = 1 * mt.exp(- 1.0*((yi-y)*(yi-y)+(xi-x)*(xi-x)))
        out = np.append(out,0)
    elif gratitude==1:
        out = np.zeros(100)
        out = np.append(out,1)

    return out.flatten()

# create tensor of the proper size (1 channel currently)
grid = torch.zeros(BATCH_SIZE,nc,2*size_padding+size_potential, 2*size_padding+size_potential) #, dtype=torch.double)

labels = torch.empty(BATCH_SIZE, dtype=torch.int64)

pos_label = torch.empty([BATCH_SIZE,2], dtype=torch.int64)
logit_label = torch.empty([BATCH_SIZE,2], dtype=torch.int64)

mat_labels = torch.empty(BATCH_SIZE, size_potential*size_potential+1, dtype=torch.float)

#def create_batch(grid=grid,labels=labels,sample_ids_real=S_id_real, real_ids_weight = None, sample_ids_random=S_id_random, return_transf=False):
def create_batch(grid=grid,mat_labels=mat_labels, pos_label=pos_label,
                 logit_label=logit_label, sample_ids_real=S_id_real, real_ids_weight = None, 
                 sample_ids_random=S_id_random, return_transf=False, trainbool = 0):

    grid = grid*0
    #labels = labels*0
    
    #x_label = x_label*0
    #y_label = y_label*0

    #pos_label = pos_label*0
    #logit_label = logit_label*0

    mat_labels = mat_labels*0
    
    if return_transf:
        transf = np.zeros(shape=(BATCH_SIZE,5))

    for b in range(BATCH_SIZE):

        if b < BATCH_SIZE_real + BATCH_SIZE_fill:
            s_id = np.random.choice(sample_ids_real, p = real_ids_weight, replace=True)
        else:
            s_id = np.random.choice(sample_ids_random)
        #print(s_id)
        # get the businesses near s_id
        mat_S = dict_S_S_mat[s_id]
        mat_I = dict_S_I_mat[s_id]
        #mat_I_restaurant = dict_S_I_restaurant_mat[s_id]  
        mat_I_cate1 = dict_S_I_cate1_mat[s_id]
        mat_I_cate2 = dict_S_I_cate2_mat[s_id]
        mat_I_cate3 = dict_S_I_cate3_mat[s_id]
        mat_I_cate4 = dict_S_I_cate4_mat[s_id]
        mat_I_cate5 = dict_S_I_cate5_mat[s_id]
        mat_I_comp = dict_S_I_comp_mat[s_id]

        if trainbool==0:
            # randomly pick rotation of this region
            theta = np.random.rand()*360

            # randomly mirror? 
            mirror_var = (np.random.rand() > 0.5)*2 - 1
                #print(mirror_var)
            # randomly pick where real store is going to be
            shift_x = np.random.rand()*cell_width*size_potential - cell_width/2
            shift_y = np.random.rand()*cell_width*size_potential - cell_width/2
            
        elif trainbool==1:
            # randomly pick rotation of this region
            theta = 0

            # randomly mirror? 
            mirror_var = 1
                #print(mirror_var)
            # randomly pick where real store is going to be
            shift_x = np.random.rand()*cell_width*size_potential - cell_width/2
            shift_y = np.random.rand()*cell_width*size_potential - cell_width/2
            

        # print(shift_x,shift_y,theta,mirror_var)
        if return_transf:
            transf[b,0] = s_id
            transf[b,1] = shift_x
            transf[b,2] = shift_y
            transf[b,3] = theta
            transf[b,4] = mirror_var

        '''
        # fill tensor
        grid[b,0,:,:] = torch.from_numpy(data_to_grid(mat_S,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
        if nc >= 2:
            grid[b,1,:,:] = torch.from_numpy(data_to_grid(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
        if nc >= 3:
            grid[b,2,:,:] = torch.from_numpy(data_to_grid(mat_I_restaurant,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
        '''

        # fill tensor
        grid[b,0,:,:] = torch.from_numpy(data_to_grid(mat_S,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
        if nc >= 2:
            grid[b,1,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=2))
            grid[b,2,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=3))
            grid[b,3,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=4))
            grid[b,4,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=5))
            grid[b,5,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=6))
            grid[b,6,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=7))
            grid[b,7,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=8))
            grid[b,8,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=9))
            grid[b,9,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=10))
            grid[b,10,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=11))
            grid[b,11,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=12))
            grid[b,12,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=13))
            grid[b,13,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=14))
            grid[b,14,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=15))
            grid[b,15,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=16))
            grid[b,16,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=17))
            grid[b,17,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=18))
            grid[b,18,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=19))

        if nc >= 20:
            grid[b,19,:,:] = torch.from_numpy(data_to_grid(mat_I_cate1,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,20,:,:] = torch.from_numpy(data_to_grid(mat_I_cate2,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,21,:,:] = torch.from_numpy(data_to_grid(mat_I_cate3,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,22,:,:] = torch.from_numpy(data_to_grid(mat_I_cate4,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,23,:,:] = torch.from_numpy(data_to_grid(mat_I_cate5,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))

        if nc >= 25:
            grid[b,24,:,:] = torch.from_numpy(data_to_grid(mat_I_comp,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))


        # include this grocery store in the covariates? (whether to gauge the point out)
        '''
        if b >= BATCH_SIZE_real and b < BATCH_SIZE_real+BATCH_SIZE_fill:
            treat_x = int(round(shift_x/cell_width) + size_padding)
            treat_y = int(round(shift_y/cell_width) + size_padding)
            grid[b,0,treat_y,treat_x] += 1
        '''
        if b <= BATCH_SIZE_real:
            treat_x = int(round(shift_x/cell_width) + size_padding)
            treat_y = int(round(shift_y/cell_width) + size_padding)
            grid[b,0,treat_y,treat_x] -= 1
        

        # location of missing grocery store:
        if b <= BATCH_SIZE_real:
            
            mat_labels[b,:] = torch.from_numpy(create_matLabel(int(round(shift_y/cell_width)), int(round(shift_x/cell_width)),0))
            
            '''
            logit_label[b,0] = float(1)
            logit_label[b,1] = float(0)
            
            #x_label[b] = int(round(shift_x/cell_width))
            #y_label[b] = int(round(shift_y/cell_width))
            
            pos_label[b,0] = float(int(round(shift_x/cell_width)))
            pos_label[b,1] = float(int(round(shift_y/cell_width)))
            '''

            #labels[b] = int(round(shift_y/cell_width)*size_potential) + int(round(shift_x/cell_width))
        # random region without missing grocery store or grocery store is filled in:
        else:
            
            mat_labels[b,:] = torch.from_numpy(create_matLabel(int(round(shift_y/cell_width)), int(round(shift_x/cell_width)), 1))
            
            '''
            logit_label[b,0] = float(0)
            logit_label[b,1] = float(1)
            
            #x_label[b] = int(round(shift_x/cell_width))
            #y_label[b] = int(round(shift_y/cell_width))
            
            pos_label[b,0] = float(int(round(shift_x/cell_width)))
            pos_label[b,1] = float(int(round(shift_y/cell_width)))

            #labels[b] = pow(size_potential,2)  # index 1 larger than locations (start at 0)
            '''
            
    if not return_transf:
        #print(labels,grid,grid.shape())
        #print(grid[60,0,:,:].sum(),grid[1,0,:,:].sum(),grid[100,0,:,:].sum())
        #print(grid[60,1,:,:].sum(),grid[1,1,:,:].sum(),grid[100,1,:,:].sum())
        
        #print(grid[60,3,:,:].sum(),grid[1,3,:,:].sum(),grid[100,3,:,:].sum())
        #print(grid[60,5,:,:].sum(),grid[1,5,:,:].sum(),grid[100,5,:,:].sum())
        #print(grid[60,7,:,:].sum(),grid[1,7,:,:].sum(),grid[100,7,:,:].sum())      
        #print(grid[60,19,:,:].sum(),grid[1,19,:,:].sum(),grid[100,19,:,:].sum())
        
        #print(list(grid[60,0,:,:]),'hihihi', list(grid[1,0,:,:]))
        #return grid, labels
        return grid, mat_labels #logit_label, pos_label
    else:
        #return grid, labels, transf
        return grid, mat_labels, transf

Define Neural Net

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        main = nn.Sequential(
            nn.InstanceNorm2d(num_features=nc, affine=True),
            nn.Conv2d(in_channels=nc,
                out_channels=2*nc,
                kernel_size=5, #9,
                padding=2, #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                bias=True),
            nn.InstanceNorm2d(num_features=2*nc, affine=True),
            nn.LeakyReLU(),
            # nn.Conv2d(in_channels=2*nc,
            #     out_channels=4*nc,
            #     kernel_size=3, #9,
            #     padding=1, #4, #(9-1)/2,
            #     padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
            #     bias=True),
            # nn.InstanceNorm2d(num_features=2*nc, affine=True),
            # nn.LeakyReLU(),
            nn.Conv2d(in_channels=2*nc,
                out_channels=4*nc,
                kernel_size=21, # 21 #9,
                padding=20,#20  #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                dilation=2,
                bias=True),
            nn.InstanceNorm2d(num_features=4*nc, affine=True),
            nn.LeakyReLU(),
            
            nn.Conv2d(in_channels=4*nc,
                out_channels=4*nc,
                kernel_size=5, #9,
                padding=2, #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                bias=True),
            nn.InstanceNorm2d(num_features=4*nc, affine=True),
            nn.LeakyReLU(),
            
            #nn.Conv2d(in_channels=4*nc,
            #    out_channels=4*nc,
            #    kernel_size=5, #9,
            #    padding=2, #4, #(9-1)/2,
            #    padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
            #    bias=True),
            #nn.InstanceNorm2d(num_features=4*nc, affine=True),
            #nn.LeakyReLU(),
            
            # nn.Conv2d(in_channels=4*nc,
            #     out_channels=2*nc,
            #     kernel_size=3, #9,
            #     padding=1, #4, #(9-1)/2,
            #     padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
            #     bias=True),
            # nn.InstanceNorm2d(num_features=2*nc, affine=True),
            # nn.LeakyReLU(),
            nn.Conv2d(in_channels=4*nc,
                out_channels=1, #1
                kernel_size=21, #9,
                padding=20, #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                dilation=2,
                bias=True),
            #nn.LeakyReLU(),
            
            #nn.Conv2d(in_channels=4*nc,
            #     out_channels=2*nc,
            #     kernel_size=3, #9,
            #     padding=1, #4, #(9-1)/2,
            #     padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
            #     bias=True),
            #nn.InstanceNorm2d(num_features=2*nc, affine=True),
            
            nn.InstanceNorm2d(num_features=1, affine=True), #1
            nn.Flatten(),  #view(-1, 1*X_dim*X_dim),
            # nn.Softmax(dim=0),  # CrossEntropyLoss already includes LogSoftmax
            nn.Linear(1*pow(2*size_padding+size_potential,2), pow(size_potential,2)+1),  #1
            #nn.Linear(1*pow(2*size_padding+size_potential,2), [pow(size_potential,2),pow(size_potential,2)]),  
        )
        #self.classifier = nn.Linear(1*pow(2*size_padding+size_potential,2), 2)
        #self.regression = nn.Linear(1*pow(2*size_padding+size_potential,2), 2)
        self.main = main

    def forward(self, x):
        output = self.main(inputs)
        #logits = self.classifier(output)
        #position = self.regression(output)
        #print(output)
        return output

function to initialize optimizer

In [18]:
def intitialize_optimizer(net):
    #return optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    #return optim.AdamW(net.parameters(), lr=0.001)
    return optim.Adam(net.parameters(), lr=0.0005)

code for saving and loading the model

In [19]:
def save_model(filename=None):
    if not filename:
        date = (datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d--%H-%M')
        filename = 'classifier-checkpoint-' + date + '.tar'
    path_save = dataroot + filename
    # save the model
    torch.save({
                # 'epoch': epoch,
                'net_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, path_save)
    print('file: ' + path_save)

def load_model(filename,net=None,optimizer=None):
    path_load = dataroot + filename
    if not net:
        net = Net()
    # if using GPU
    if use_cuda and torch.cuda.is_available():
        net.cuda()
    if not optimizer:
        optimizer = intitialize_optimizer(net)
    checkpoint = torch.load(path_load)
    net.load_state_dict(checkpoint['net_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])



    return net, optimizer

Set random seed

In [20]:
# Set random seed for reproducibility
manualSeed = 24601
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  24601


Pick training / evaluation sample

In [21]:
# locations for training and evaluation

num_distinct_train_real = int(len(S_id_real) * frac_train_real)
#num_distinct_train_real = int(numbSidFine * frac_train_real)

num_distinct_train_random = int(len(S_id_random) * frac_train_random)

#TODO: implement such that eval regions are non-overlapping with train regions
SidRealWeight = []
for i in S_id_real:
    if i<=(fineSampleNumber-1):
        SidRealWeight.append(weight_fine)
    else:
        SidRealWeight.append(weight_plain)
        
SidRealWeight = np.array(SidRealWeight)
SidRealWeight = SidRealWeight/np.sum(SidRealWeight)

sample_train_real = list(np.random.choice(a=S_id_real, size=num_distinct_train_real, replace=True, p = SidRealWeight))
sample_train_real.sort()


sample_train_random = list(np.random.choice(a=S_id_random,size=num_distinct_train_random,replace=False))
sample_train_random.sort()

if num_distinct_train_real < len(S_id_real):
    sample_eval_real = list(set(S_id_real) - set(sample_train_real))
else:
    sample_eval_real = S_id_real
sample_eval_real.sort()

final_eval_real = [item for item in S_id_real if item<= (fineSampleNumber-1)]


# weights
train_id_weight = []
for i in sample_train_real:
    if i<=(fineSampleNumber-1):
        train_id_weight.append(weight_fine)
    else:
        train_id_weight.append(weight_plain)
        
train_id_weight = np.array(train_id_weight)
train_id_weight = train_id_weight/np.sum(train_id_weight)

eval_id_weight = []
for i in sample_eval_real:
    if i<=(fineSampleNumber-1):
        eval_id_weight.append(eval_weight_fine)
    else:
        eval_id_weight.append(eval_weight_plain)
        
eval_id_weight = np.array(eval_id_weight)
eval_id_weight = eval_id_weight/np.sum(eval_id_weight)
        
finalEval_id_weight = []
for i in final_eval_real:
    if i<=(fineSampleNumber-1):
        finalEval_id_weight.append(eval_weight_fine)
    else:
        finalEval_id_weight.append(eval_weight_plain)
        
finalEval_id_weight = np.array(finalEval_id_weight)
finalEval_id_weight = finalEval_id_weight/np.sum(finalEval_id_weight)
        

if num_distinct_train_random < len(S_id_random):
    sample_eval_random = list(set(S_id_random) - set(sample_train_random))
else:
    sample_eval_random = S_id_random
sample_eval_random.sort()

Initialize neural net

In [24]:
def weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    #print('0')
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
        #print('h')
    elif classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


if use_saved_model:
    net, optimizer = load_model(saved_model_filename)
else:
    net = Net()
    net.apply(weights_init)
    if use_cuda and torch.cuda.is_available():
        net.cuda()
    optimizer = intitialize_optimizer(net)

criterion = nn.CrossEntropyLoss()
'''
class my_loss(nn.Module):
    def __init__(self, alpha=3.0):
        super().__init__()
        self.cls_loss = nn.CrossEntropyLoss()
        self.pos_loss = nn.MSELoss(reduction="none")
        self.alpha = alpha
        
    def forward(self, logits, position, logit_label, pos_label):
        #print(logits, logit_label)
        #print(position)
        cls_loss = self.cls_loss(logits, logit_label.float())
        pos_loss = self.pos_loss(position, pos_label.float()).sum(dim=-1)
        pos_loss = torch.where(logit_label[:, 0] == 0, pos_loss*0, pos_loss)
        return cls_loss + self.alpha * pos_loss.mean()

criterion = my_loss()
'''   
    
if use_cuda and torch.cuda.is_available():
    net.cuda()

In [25]:
count = 0
for k, v in net.named_parameters():
    count += v.numel()
print(count)

2783754


Training

In [26]:
print('Starting Training')
print((datetime.utcnow() + timedelta(hours=-4)).strftime('%Y-%m-%d %H:%M:%S'))

# loop over the dataset multiple times
print('epoch:',EPOCHS, 'iteration:', ITERS)
for epoch in range(EPOCHS):

    # initialize fit statistics
    '''
    running_loss = 0.0
    logit_tot_correct = 0
    logit_real_correct = 0
    posX_real_correct = 0
    posY_real_correct = 0
    real_correct = 0
    logit_fill_correct = 0
    logit_random_correct = 0
    total = 0
    total_real = 0
    total_fill = 0
    total_random = 0
    '''
    running_loss = 0.0
    correct = 0
    correct_real = 0
    non_zero_real = 0
    correct_fill = 0
    correct_random = 0
    total = 0
    total_real = 0
    total_fill = 0
    total_random = 0
    cnt_real_about = 0


    for i in range(ITERS):
        #print(epoch,i)
        # get the inputs; data is a list of [inputs, labels]
        data = create_batch(sample_ids_real=sample_train_real, real_ids_weight = train_id_weight,
                            sample_ids_random=sample_train_random, trainbool=0)
        inputs, labels = data
        #inputs, logit_label, pos_label = data

        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
            #logit_label = logit_label.cuda()
            #pos_label = pos_label.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        '''
        logits, position = net(inputs)
        loss = criterion(logits, position, logit_label, pos_label)
        loss.backward()
        optimizer.step()
        '''
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # determine accuracy of taking "prediction"
        with torch.no_grad():
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels, 1)
            #_, predicted = torch.max(logits.data, 1)

            for k in range(len(predicted[0:BATCH_SIZE_real])):
                judge = predicted[k]-labels[k]
                if judge==1 or judge==-1 or judge==10 or judge==-10 or judge==11 or judge==-11 or judge==9 or judge==-9 or judge==0:
                    cnt_real_about+=1

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            correct_real += (predicted[0:BATCH_SIZE_real] == labels[0:BATCH_SIZE_real]).sum().item()
            non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
            correct_fill += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == labels[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill]).sum().item()
            correct_random += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == labels[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE]).sum().item()
            total_real += BATCH_SIZE_real
            total_fill += BATCH_SIZE_fill
            total_random += BATCH_SIZE_random
            
            '''
            total += logit_label.size(0)
            logit_tot_correct += (predicted == logit_label[:,0]).sum().item()
            
            logit_real_correct += (predicted[0:BATCH_SIZE_real] == logit_label[0:BATCH_SIZE_real,0]).sum().item()
            posX_real_correct += (position.int()[0:BATCH_SIZE_real,0] == pos_label[0:BATCH_SIZE_real,0]).sum().item()
            posY_real_correct += (position.int()[0:BATCH_SIZE_real,1] == pos_label[0:BATCH_SIZE_real,1]).sum().item()
            
            a = (predicted[0:BATCH_SIZE_real] == logit_label[0:BATCH_SIZE_real,0])
            b = (position.int()[0:BATCH_SIZE_real,0] == pos_label[0:BATCH_SIZE_real,0])
            c = (position.int()[0:BATCH_SIZE_real,1] == pos_label[0:BATCH_SIZE_real,1])
            a_b = torch.logical_and(a,b)
            real_correct += torch.logical_and(a_b,c).sum().item()
            #non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
            logit_fill_correct += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == logit_label[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill,0]).sum().item()
            logit_random_correct += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == logit_label[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE,0]).sum().item()
            
            total_real += BATCH_SIZE_real
            total_fill += BATCH_SIZE_fill
            total_random += BATCH_SIZE_random
            '''

        # print statistics
        running_loss += loss.item()
        if i % min(10000,ITERS/10) == min(10000,ITERS/10)-1:    # print every min(1000,ITERS/10) mini-batches
            print('[%d / %d, %5d / %5d] loss: %.3f, accuracy: %.1f%%, real: %.1f%%, real proxy: %.1f%%, real non-zero: %.1f%%, real filled: %.1f%%, unrealized: %.1f%%' %
                  (epoch + 1, EPOCHS, i + 1, ITERS, running_loss / min(10000,ITERS/10),
                   100 * correct / total,
                   100 * correct_real / max(total_real,1),
                   100 * cnt_real_about / max(total_real,1),
                   100 * non_zero_real / max(total_real,1),
                   100 * correct_fill / max(total_fill,1),
                   100 * correct_random / max(total_random,1)))
            running_loss = 0.0
            correct = 0
            correct_real = 0
            non_zero_real = 0
            correct_fill = 0
            correct_random = 0
            total = 0
            total_real = 0
            total_fill = 0
            total_random = 0
            cnt_real_about = 0

        '''
        running_loss += loss.item()
        if i % min(10000,ITERS/10) == min(10000,ITERS/10)-1:    # print every min(1000,ITERS/10) mini-batches
            print('[%d / %d, %5d / %5d] loss: %.3f, totAcc: %.1f%%, totLogitAcc: %.1f%%, realAcc: %.1f%%, realLogitAcc: %.1f%%, fillAcc: %.1f%%, unrealizedAcc: %.1f%%' %
                  (epoch + 1, EPOCHS, i + 1, ITERS, running_loss / min(10000,ITERS/10),
                   100 * (logit_fill_correct+logit_random_correct+real_correct)/total,
                   100 * logit_tot_correct / total,
                   
                   100 * real_correct / max(total_real,1),
                   100 * logit_real_correct / max(total_real,1),
                   100 * logit_fill_correct / max(total_fill,1),
                   100 * logit_random_correct / max(total_random,1)))
            running_loss = 0.0
            logit_tot_correct = 0
            logit_real_correct = 0
            posX_real_correct = 0
            posY_real_correct = 0
            real_correct = 0
            logit_fill_correct = 0
            logit_random_correct = 0
            total = 0
            total_real = 0
            total_fill = 0
            total_random = 0
        '''
            
        # evaluation sample:
        if frac_train_real < 1 or frac_train_random < 1:
            if i % min(10000,ITERS/2) == min(10000,ITERS/2)-1:    # print every min(1000,ITERS/10) mini-batches
                '''
                eval_correct = 0
                eval_correct_real = 0
                eval_non_zero_real = 0
                eval_correct_fill = 0
                eval_correct_random = 0
                eval_total = 0
                eval_total_real = 0
                eval_total_fill = 0
                eval_total_random = 0
                
                running_loss = 0.0
                '''
                eval_correct = 0
                eval_correct_real = 0
                eval_non_zero_real = 0
                eval_correct_fill = 0
                eval_correct_random = 0
                eval_total = 0
                eval_total_real = 0
                eval_total_fill = 0
                eval_total_random = 0
                eval_cnt_real_about = 0

                with torch.no_grad():
                    for i in range(100):
                        inputs, labels = create_batch(sample_ids_real=sample_eval_real, real_ids_weight = eval_id_weight,
                                                    sample_ids_random=sample_eval_random, trainbool=1)

                        if use_cuda and torch.cuda.is_available():
                            inputs = inputs.cuda()
                            labels = labels.cuda()

                        outputs = net(inputs)
                        _, predicted = torch.max(outputs.data, 1)
                        _, labels = torch.max(labels, 1)
                        
                        for k in range(len(predicted[0:BATCH_SIZE_real])):
                            judge = predicted[k]-labels[k]
                            if judge==1 or judge==-1 or judge==10 or judge==-10 or judge==11 or judge==-11 or judge==9 or judge==-9 or judge==0:
                                eval_cnt_real_about+=1

                        eval_total += labels.size(0)
                        eval_correct += (predicted == labels).sum().item()
                        eval_correct_real += (predicted[0:BATCH_SIZE_real] == labels[0:BATCH_SIZE_real]).sum().item()
                        eval_non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
                        eval_correct_fill += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == labels[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill]).sum().item()
                        eval_correct_random += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == labels[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE]).sum().item()
                        eval_total_real += BATCH_SIZE_real
                        eval_total_fill += BATCH_SIZE_fill
                        eval_total_random += BATCH_SIZE_random
                        
                        '''
                        eval_total += labels.size(0)
                        eval_correct += (predicted == labels).sum().item()
                        eval_correct_real += (predicted[0:BATCH_SIZE_real] == labels[0:BATCH_SIZE_real]).sum().item()
                        eval_non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
                        eval_correct_fill += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == labels[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill]).sum().item()
                        eval_correct_random += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == labels[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE]).sum().item()
                        eval_total_real += BATCH_SIZE_real
                        eval_total_fill += BATCH_SIZE_fill
                        eval_total_random += BATCH_SIZE_random
                        '''
                print('Accuracy on hold-out: %.1f%%, real: %.1f%%, real proxy: %.1f%%, real non-zero: %.1f%%, real filled: %.1f%%, unrealized: %.1f%%' %
                    (100 * eval_correct / max(eval_total,1),
                    100 * eval_correct_real / max(eval_total_real,1),
                    100 * eval_cnt_real_about / max(eval_total_real,1),
                    100 * eval_non_zero_real / max(eval_total_real,1),
                    100 * eval_correct_fill / max(eval_total_fill,1),
                    100 * eval_correct_random / max(eval_total_random,1)))


    print('Finished Epoch ' + str(epoch+1) + ' of ' + str(EPOCHS) + '. Saving model and optimizer checkpoint.')
    save_model()
    print((datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d %H:%M:%S'))

print('Finished Training')

Starting Training
2022-08-31 00:38:57
epoch: 10 iteration: 100
[1 / 10,    10 /   100] loss: 11.733, accuracy: 6.8%, real: 1.6%, real proxy: 10.7%, real non-zero: 76.2%, real filled: 28.1%, unrealized: 26.2%
[1 / 10,    20 /   100] loss: 10.471, accuracy: 11.2%, real: 8.4%, real proxy: 27.0%, real non-zero: 75.5%, real filled: 28.8%, unrealized: 10.0%
[1 / 10,    30 /   100] loss: 8.540, accuracy: 30.8%, real: 34.1%, real proxy: 61.6%, real non-zero: 85.8%, real filled: 17.5%, unrealized: 17.5%
[1 / 10,    40 /   100] loss: 7.166, accuracy: 48.8%, real: 55.6%, real proxy: 80.9%, real non-zero: 92.7%, real filled: 19.4%, unrealized: 26.2%
[1 / 10,    50 /   100] loss: 6.521, accuracy: 58.7%, real: 60.9%, real proxy: 83.6%, real non-zero: 94.2%, real filled: 51.9%, unrealized: 45.0%
[1 / 10,    60 /   100] loss: 6.087, accuracy: 69.8%, real: 65.5%, real proxy: 87.3%, real non-zero: 96.0%, real filled: 95.6%, unrealized: 68.8%
[1 / 10,    70 /   100] loss: 5.992, accuracy: 71.0%, real: 65

[6 / 10,    10 /   100] loss: 5.204, accuracy: 89.1%, real: 86.8%, real proxy: 94.4%, real non-zero: 99.4%, real filled: 98.8%, unrealized: 97.5%
[6 / 10,    20 /   100] loss: 5.157, accuracy: 88.1%, real: 85.4%, real proxy: 94.5%, real non-zero: 99.5%, real filled: 100.0%, unrealized: 96.2%
[6 / 10,    30 /   100] loss: 5.191, accuracy: 86.9%, real: 84.8%, real proxy: 94.3%, real non-zero: 99.4%, real filled: 98.8%, unrealized: 88.8%
[6 / 10,    40 /   100] loss: 5.062, accuracy: 87.8%, real: 85.3%, real proxy: 94.9%, real non-zero: 99.2%, real filled: 99.4%, unrealized: 95.0%
[6 / 10,    50 /   100] loss: 5.100, accuracy: 88.9%, real: 86.6%, real proxy: 95.0%, real non-zero: 99.4%, real filled: 99.4%, unrealized: 96.2%
[6 / 10,    60 /   100] loss: 5.169, accuracy: 88.7%, real: 86.1%, real proxy: 95.0%, real non-zero: 99.2%, real filled: 99.4%, unrealized: 97.5%
[6 / 10,    70 /   100] loss: 5.165, accuracy: 87.9%, real: 86.0%, real proxy: 94.3%, real non-zero: 99.7%, real filled: 98

Evaluation

In [17]:
print('Starting Evaluation')

eval_correct = 0
eval_correct_real = 0
eval_non_zero_real = 0
eval_correct_fill = 0
eval_correct_random = 0
eval_total = 0
eval_total_real = 0
eval_total_fill = 0
eval_total_random = 0

with torch.no_grad():
    for i in range(1000):
        print(i)
        inputs, labels = create_batch(sample_ids_real=sample_eval_real, real_ids_weight = finalEval_id_weight,
                                        sample_ids_random=sample_eval_random)

        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels, 1)

        eval_total += labels.size(0)
        
        eval_correct += (predicted == labels).sum().item()
        eval_correct_real += (predicted[0:BATCH_SIZE_real] == labels[0:BATCH_SIZE_real]).sum().item()
        eval_non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
        eval_correct_fill += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == labels[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill]).sum().item()
        eval_correct_random += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == labels[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE]).sum().item()
        
        eval_total_real += BATCH_SIZE_real
        eval_total_fill += BATCH_SIZE_fill
        eval_total_random += BATCH_SIZE_random

print('Accuracy on hold-out: %.1f%%, real: %.1f%%, real non-zero: %.1f%%, real filled: %.1f%%, unrealized: %.1f%%' %
        (100 * eval_correct / max(eval_total,1),
        100 * eval_correct_real / max(eval_total_real,1),
        100 * eval_non_zero_real / max(eval_total_real,1),
        100 * eval_correct_fill / max(eval_total_fill,1),
        100 * eval_correct_random / max(eval_total_random,1)))

print((datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d %H:%M:%S'))

Starting Evaluation
0
tensor(1.8570) tensor(0.6380) tensor(0.)
tensor(1.7144) tensor(0.3340) tensor(0.)
tensor(298791.) tensor(38333.) tensor(0.)
tensor(18.) tensor(1.) tensor(0.)
1
tensor(3.1086) tensor(0.8333) tensor(1.1510)
tensor(2.7208) tensor(0.6682) tensor(0.7174)
tensor(344918.) tensor(49028.) tensor(85015.)
tensor(2.) tensor(6.) tensor(4.)
2
tensor(1.4970) tensor(0.) tensor(0.6397)
tensor(0.9462) tensor(0.) tensor(0.6670)
tensor(154268.) tensor(0.) tensor(153417.)
tensor(5.) tensor(0.) tensor(1.)
3
tensor(1.2105) tensor(0.) tensor(0.)
tensor(1.0717) tensor(0.) tensor(0.)
tensor(69480.) tensor(0.) tensor(0.)
tensor(5.) tensor(7.) tensor(0.)
4
tensor(0.7339) tensor(1.8436) tensor(0.)
tensor(0.4698) tensor(1.1097) tensor(0.)
tensor(43447.) tensor(638960.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
5
tensor(6.8337) tensor(0.6187) tensor(1.2475)
tensor(5.5930) tensor(0.4317) tensor(1.2083)
tensor(709569.) tensor(90313.) tensor(239000.)
tensor(0.) tensor(0.) tensor(28.)
6
tensor(0.

53
tensor(0.6105) tensor(2.3846) tensor(0.)
tensor(0.2953) tensor(2.6051) tensor(0.)
tensor(72000.) tensor(381073.) tensor(0.)
tensor(3.) tensor(3.) tensor(1.)
54
tensor(3.7689) tensor(0.) tensor(0.6622)
tensor(2.1587) tensor(0.) tensor(0.6490)
tensor(543395.) tensor(0.) tensor(78462.)
tensor(0.) tensor(1.) tensor(0.)
55
tensor(0.) tensor(0.) tensor(2.3486)
tensor(0.) tensor(0.) tensor(1.7493)
tensor(0.) tensor(0.) tensor(239972.)
tensor(5.) tensor(0.) tensor(23.)
56
tensor(0.) tensor(1.6824) tensor(1.1807)
tensor(0.) tensor(1.2120) tensor(0.8253)
tensor(0.) tensor(178059.) tensor(34502.)
tensor(6.) tensor(101.) tensor(2.)
57
tensor(0.) tensor(0.6634) tensor(3.3754)
tensor(0.) tensor(0.4568) tensor(2.4446)
tensor(0.) tensor(93443.) tensor(509029.)
tensor(8.) tensor(0.) tensor(0.)
58
tensor(0.) tensor(2.5232) tensor(0.6494)
tensor(0.) tensor(2.3308) tensor(0.5992)
tensor(0.) tensor(209850.) tensor(90585.)
tensor(13.) tensor(5.) tensor(1.)
59
tensor(0.) tensor(0.) tensor(4.5551)
tensor(0

106
tensor(0.5990) tensor(2.1705) tensor(7.3759)
tensor(0.3875) tensor(1.7327) tensor(4.7892)
tensor(33917.) tensor(172070.) tensor(400284.)
tensor(0.) tensor(0.) tensor(0.)
107
tensor(1.3368) tensor(0.) tensor(0.6067)
tensor(1.0589) tensor(0.) tensor(0.6605)
tensor(195877.) tensor(0.) tensor(122800.)
tensor(0.) tensor(0.) tensor(2.)
108
tensor(2.0786) tensor(0.) tensor(9.7750)
tensor(1.8638) tensor(0.) tensor(3.1666)
tensor(291932.) tensor(0.) tensor(527789.)
tensor(0.) tensor(0.) tensor(0.)
109
tensor(0.) tensor(0.) tensor(0.6634)
tensor(0.) tensor(0.) tensor(0.4568)
tensor(0.) tensor(0.) tensor(93443.)
tensor(0.) tensor(0.) tensor(0.)
110
tensor(6.3303) tensor(0.) tensor(0.)
tensor(4.7267) tensor(0.) tensor(0.)
tensor(622850.) tensor(0.) tensor(0.)
tensor(48.) tensor(0.) tensor(0.)
111
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(19.) tensor(0.)
112
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0

tensor(5.9821) tensor(0.) tensor(0.)
tensor(4.1969) tensor(0.) tensor(0.)
tensor(358841.) tensor(0.) tensor(0.)
tensor(0.) tensor(22.) tensor(10.)
159
tensor(0.5986) tensor(6.7578) tensor(1.2515)
tensor(0.5137) tensor(5.5394) tensor(1.0915)
tensor(42202.) tensor(740730.) tensor(80115.)
tensor(0.) tensor(0.) tensor(23.)
160
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(23.) tensor(0.)
161
tensor(2.6494) tensor(1.1543) tensor(0.)
tensor(2.3179) tensor(0.3961) tensor(0.)
tensor(305776.) tensor(57465.) tensor(0.)
tensor(8.) tensor(3.) tensor(0.)
162
tensor(0.) tensor(2.6055) tensor(0.)
tensor(0.) tensor(2.6345) tensor(0.)
tensor(0.) tensor(287515.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
163
tensor(0.) tensor(0.7164) tensor(0.)
tensor(0.) tensor(0.5054) tensor(0.)
tensor(0.) tensor(49211.) tensor(0.)
tensor(9.) tensor(0.) tensor(0.)
164
tensor(0.) tensor(0.5621) tensor(2.5364)
tensor(0.) tensor(0.5758) tensor(2.0859

211
tensor(1.2142) tensor(0.7321) tensor(4.5476)
tensor(1.0406) tensor(0.4923) tensor(1.8912)
tensor(182516.) tensor(35703.) tensor(222724.)
tensor(6.) tensor(0.) tensor(0.)
212
tensor(1.3480) tensor(0.) tensor(0.)
tensor(1.0902) tensor(0.) tensor(0.)
tensor(167497.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
213
tensor(5.9016) tensor(0.) tensor(1.5190)
tensor(3.5612) tensor(0.) tensor(1.0793)
tensor(499662.) tensor(0.) tensor(157107.)
tensor(2.) tensor(3.) tensor(0.)
214
tensor(0.5302) tensor(0.) tensor(0.)
tensor(0.4647) tensor(0.) tensor(0.)
tensor(40813.) tensor(0.) tensor(0.)
tensor(3.) tensor(9.) tensor(0.)
215
tensor(1.1635) tensor(0.) tensor(0.)
tensor(0.9772) tensor(0.) tensor(0.)
tensor(75940.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(8.)
216
tensor(0.6347) tensor(0.) tensor(1.3705)
tensor(0.6256) tensor(0.) tensor(1.0594)
tensor(56767.) tensor(0.) tensor(145807.)
tensor(0.) tensor(9.) tensor(5.)
217
tensor(0.) tensor(0.7866) tensor(0.5615)
tensor(0.) te

...

739
tensor(0.7032) tensor(2.8240) tensor(0.)
tensor(0.5242) tensor(2.5175) tensor(0.)
tensor(79419.) tensor(394198.) tensor(0.)
tensor(0.) tensor(0.) tensor(4.)
740
tensor(0.3521) tensor(1.2018) tensor(0.)
tensor(0.4067) tensor(0.6762) tensor(0.)
tensor(44954.) tensor(59397.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
741
tensor(3.3796) tensor(0.5583) tensor(0.)
tensor(2.6394) tensor(0.5269) tensor(0.)
tensor(349558.) tensor(42500.) tensor(0.)
tensor(6.) tensor(0.) tensor(0.)
742
tensor(1.8950) tensor(0.6100) tensor(0.)
tensor(1.6127) tensor(0.4743) tensor(0.)
tensor(301529.) tensor(50938.) tensor(0.)
tensor(23.) tensor(24.) tensor(10.)
743
tensor(1.8720) tensor(0.) tensor(0.)
tensor(1.2314) tensor(0.) tensor(0.)
tensor(101937.) tensor(0.) tensor(0.)
tensor(3.) tensor(13.) tensor(0.)
744
tensor(0.) tensor(0.5489) tensor(0.6617)
tensor(0.) tensor(0.5452) tensor(0.5943)
tensor(0.) tensor(100139.) tensor(81563.)
tensor(0.) tensor(11.) tensor(0.)
745
tensor(6.5927) tensor(2.1705) tensor(0

792
tensor(0.5871) tensor(0.) tensor(1.1990)
tensor(0.4536) tensor(0.) tensor(1.3108)
tensor(50556.) tensor(0.) tensor(161517.)
tensor(1.) tensor(0.) tensor(0.)
793
tensor(0.) tensor(2.8790) tensor(0.6463)
tensor(0.) tensor(1.7906) tensor(0.5932)
tensor(0.) tensor(259827.) tensor(56125.)
tensor(1.) tensor(8.) tensor(3.)
794
tensor(0.5978) tensor(0.) tensor(0.7109)
tensor(0.7711) tensor(0.) tensor(0.5259)
tensor(91711.) tensor(0.) tensor(86571.)
tensor(9.) tensor(9.) tensor(0.)
795
tensor(3.8786) tensor(0.6132) tensor(0.)
tensor(3.2395) tensor(0.3757) tensor(0.)
tensor(522295.) tensor(38333.) tensor(0.)
tensor(0.) tensor(1.) tensor(2.)
796
tensor(1.1635) tensor(0.6152) tensor(0.5725)
tensor(0.9772) tensor(0.3545) tensor(0.6300)
tensor(75940.) tensor(61346.) tensor(124886.)
tensor(1.) tensor(0.) tensor(0.)
797
tensor(0.5725) tensor(5.0823) tensor(0.)
tensor(0.6300) tensor(3.2011) tensor(0.)
tensor(124886.) tensor(496595.) tensor(0.)
tensor(0.) tensor(2.) tensor(0.)
798
tensor(0.7989) ten

844
tensor(0.4856) tensor(0.) tensor(0.7513)
tensor(0.2930) tensor(0.) tensor(0.5637)
tensor(21875.) tensor(0.) tensor(31108.)
tensor(3.) tensor(0.) tensor(0.)
845
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(5.) tensor(8.)
846
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(6.)
847
tensor(0.6628) tensor(1.4446) tensor(1.3080)
tensor(0.5827) tensor(0.6826) tensor(0.6821)
tensor(78846.) tensor(150893.) tensor(111845.)
tensor(0.) tensor(3.) tensor(0.)
848
tensor(0.6349) tensor(0.) tensor(0.)
tensor(0.6004) tensor(0.) tensor(0.)
tensor(149716.) tensor(0.) tensor(0.)
tensor(7.) tensor(16.) tensor(21.)
849
tensor(0.8705) tensor(0.) tensor(0.)
tensor(0.5076) tensor(0.) tensor(0.)
tensor(60969.) tensor(0.) tensor(0.)
tensor(5.) tensor(0.) tensor(8.)
850
tensor(0.) tensor(1.2640) tensor(0.)
tensor(0.) tensor(0.2665) tensor(0.)
tensor(0.) tensor

897
tensor(3.7689) tensor(0.8894) tensor(0.)
tensor(2.1587) tensor(1.0401) tensor(0.)
tensor(543395.) tensor(105402.) tensor(0.)
tensor(0.) tensor(21.) tensor(0.)
898
tensor(0.6463) tensor(1.9491) tensor(0.6559)
tensor(0.5932) tensor(1.3111) tensor(0.5940)
tensor(56125.) tensor(164230.) tensor(50078.)
tensor(3.) tensor(9.) tensor(0.)
899
tensor(0.) tensor(0.) tensor(0.5489)
tensor(0.) tensor(0.) tensor(0.3703)
tensor(0.) tensor(0.) tensor(33472.)
tensor(9.) tensor(0.) tensor(0.)
900
tensor(5.0100) tensor(0.) tensor(0.)
tensor(3.2695) tensor(0.) tensor(0.)
tensor(293131.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
901
tensor(1.3517) tensor(0.) tensor(0.7338)
tensor(1.0368) tensor(0.) tensor(0.5941)
tensor(191333.) tensor(0.) tensor(82875.)
tensor(22.) tensor(4.) tensor(0.)
902
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(9.)
903
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)

950
tensor(0.5707) tensor(0.7676) tensor(0.)
tensor(0.2588) tensor(0.6512) tensor(0.)
tensor(27841.) tensor(103854.) tensor(0.)
tensor(0.) tensor(64.) tensor(4.)
951
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(6.)
952
tensor(0.) tensor(0.) tensor(1.2169)
tensor(0.) tensor(0.) tensor(1.1278)
tensor(0.) tensor(0.) tensor(241749.)
tensor(22.) tensor(41.) tensor(0.)
953
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
tensor(0.) tensor(0.) tensor(0.)
954
tensor(0.) tensor(0.7392) tensor(0.5674)
tensor(0.) tensor(0.3934) tensor(0.4688)
tensor(0.) tensor(48239.) tensor(65455.)
tensor(0.) tensor(5.) tensor(11.)
955
tensor(0.6872) tensor(0.) tensor(1.2991)
tensor(0.5687) tensor(0.) tensor(1.2111)
tensor(71953.) tensor(0.) tensor(176328.)
tensor(0.) tensor(0.) tensor(0.)
956
tensor(0.) tensor(1.2829) tensor(0.5935)
tensor(0.) tensor(1.0253) tensor(0.5543)
tensor(

Functions to transform output into list for saving to file

In [27]:
@numba.jit
def data_reverse_shift_rotate(xy,shift_x=0,shift_y=0,theta=0,mirror_var=1):
    # reverse shift
    xy[:,0] -= shift_x
    xy[:,1] -= shift_y

    # reverse mirroring
    if mirror_var == 1 or mirror_var == -1:
        xy[:,0] = mirror_var * xy[:,0]
    
    # reverse rotation by theta
    theta = theta * pi / 180
    if not theta == 0:
        # x = cos(theta) * mat[:,0] - sin(theta) * mat[:,1]
        # y = sin(theta) * mat[:,0] + cos(theta) * mat[:,1]
        rot = np.array([[cos(theta),sin(theta)],[-sin(theta),cos(theta)]])
        xy = xy@np.linalg.inv(rot)
        # x = xy[:,0]
        # y = xy[:,1]
    return xy


def add_to_list(xy,o,r):
    for i in range(len(xy)):
        if i == len(xy) - 1:
            tup = (str(int(round(xy[i,0]))), str(r), 'NA', 'NA', str(o[i]))
        else:
            tup = (str(int(round(xy[i,0]))), str(r), str(int(round(xy[i,1]))), str(int(round(xy[i,2]))), str(o[i]))
        list_out.append(tup)


def outputs_to_loc(outputs,transf):
    o = outputs.cpu().numpy()
    g = np.linspace(start=cell_width/2,
                    stop=cell_width/2 + cell_width*size_potential,
                    num=size_potential, endpoint=False)
    
    for b in range(BATCH_SIZE):
        # grid cell midpoints
        xy = np.zeros(shape=(pow(size_potential,2)+1,3))
        # set s_id
        xy[:,0] = int(transf[b,0])
        # set relative location
        xy[0:pow(size_potential,2),1] = np.tile(g,size_potential)
        xy[0:pow(size_potential,2),2] = np.repeat(g,size_potential)

        xy[0:pow(size_potential,2),1:3] = data_reverse_shift_rotate(xy[0:pow(size_potential,2),1:3],
                                                                    shift_x=transf[b,1],
                                                                    shift_y=transf[b,2],
                                                                    theta=transf[b,3],
                                                                    mirror_var=transf[b,4])
        add_to_list(xy,o[b,:],b < BATCH_SIZE_real)


run the neural net on many practice examples to guess real locations

In [28]:
list_out = list()

num_batches_predict = 600 # could be more to offer more choices

# change ratio of what regions to simulate since we don't care much about the real ones
BATCH_SIZE_real = 2  # regions with missing grocery store per batch #2
BATCH_SIZE_fill = 2  # regions with real location filled in (-> no missing) per batch #2
BATCH_SIZE_random = 28  # random regions (-> no missing) per batch


finalEval_id_weight = []
for i in final_eval_real:
    if i<=int(fineSampleNumber/2-1):
        finalEval_id_weight.append(1)
    else:
        finalEval_id_weight.append(1)

        
finalEval_id_weight = np.array(finalEval_id_weight)
finalEval_id_weight = finalEval_id_weight/np.sum(finalEval_id_weight)        
        

with torch.no_grad():
    for i in range(num_batches_predict):
        print(i)
        # get the inputs; data is a list of [inputs, labels]
        data = create_batch(sample_ids_real=final_eval_real,real_ids_weight = finalEval_id_weight, return_transf=True)
        inputs, labels, transf = data
        #_, labels = torch.max(labels, 1)
        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        outputs_to_loc(outputs,transf)

# print(list_out)

0


<ipython-input-27-1b32f8becce2>:17: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, F))
  xy = xy@np.linalg.inv(rot)
C:\Users\chensimon\AppData\Local\conda\conda\envs\try3\lib\site-packages\numba\core\typing\npydecl.py:930: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, F))
  warnings.warn(NumbaPerformanceWarning(msg))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [29]:
print(len(list_out))

7272000


Save the resulting file


In [30]:
date = (datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d--%H-%M')
filename_out = 'predicted_activation-' + date + '.csv'
with open(dataroot+filename_out,'w') as f:
    f.write('S_id,real_missing,x,y,activation\n')
    for e in list_out:
        f.write(','.join(e) + '\n')